In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
from pathlib import Path

from dotenv import load_dotenv

# The notebook should be executed from the project root directory
if "_correct_path" not in locals():
    os.chdir("..")
    sys.path.append(".")
    print(f"changed dir to {Path('.').resolve()})")
    _correct_path = True
load_dotenv()

In [ ]:
import json

import pandas as pd

from docsassist.schema import RAGInput

data = RAGInput(
    promptText="What is DataRobot?", association_id="42", messages=[]
).model_dump()
data["messages"] = json.dumps(data["messages"])

rag_input = pd.DataFrame.from_records([data])
rag_input

### Get predictions as pd.Dataframe using the `score` endpoint

In [ ]:
import datarobot as dr
from datarobot_predict.deployment import predict

from docsassist.deployments import RAGDeployment

rag_deployment_id = RAGDeployment().id

predict(dr.Deployment.get(rag_deployment_id), rag_input).dataframe

### Get predictions as openai ChatCompletion using the `chat` endpoint

In [ ]:
import datarobot as dr
from IPython.display import Markdown as md
from IPython.display import clear_output
from openai import OpenAI

dr_client = dr.client.get_client()
openai_client = OpenAI(
    base_url=dr_client.endpoint + f"/deployments/{rag_deployment_id}",
    api_key=dr_client.token,
)
response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What metrics does DataRobot support? Please return a list",
        }
    ],
    model="llm-blueprint",
)

md(response.choices[0].message.content)

In [ ]:
response.citations

### Make predictions using the helper function

In [ ]:
from docsassist.predict import get_rag_completion

prediction, association_id = get_rag_completion(
    "How is DataRobot so convenient?", messages=[]
)

In [ ]:
md(prediction.completion)

In [ ]:
prediction.references

# Test the model locally

In [ ]:
sys.path.append("deployment_diy_rag")
from deployment_diy_rag.custom import chat, load_model, score

### Use the chain defined in `deployment_diy_rag/custom.py` directly

In [ ]:
chain = load_model("deployment_diy_rag")

In [ ]:
stream = chain.stream(
    input={
        "input": "What is a playground? Please respond with a very detailed long exhibition of what can be done.",
        "chat_history": [],
    }
)

answer = ""
for chunk in stream:
    if "answer" in chunk:
        answer += chunk["answer"]
        clear_output(wait=True)
        display(md(answer))

### Use the score hook directly

In [ ]:
import json

import pandas as pd

from docsassist.schema import PROMPT_COLUMN_NAME, TARGET_COLUMN_NAME

data = pd.DataFrame.from_records(
    [
        {
            PROMPT_COLUMN_NAME: "How would I get one deployed?",
            "messages": json.dumps(  # we provide a history here
                [
                    {
                        "role": "user",
                        "content": "What is a Playground",
                    },
                    {
                        "role": "assistant",
                        "content": "A Playground can be used to create LLM's inside DataRobot",
                    },
                ]
            ),
        }
    ]
)
response = score(data, model=load_model("deployment_diy_rag"))
md(response[TARGET_COLUMN_NAME].values[0])

In [ ]:
response

### use the `chat` hook directly

In [ ]:
params = {
    "messages": [{"role": "user", "content": "What is a playground?"}],
    "model": "llm-blueprint",
    "stream": True,
}

response = chat(params, load_model("deployment_diy_rag"))

md(response.choices[0].message.content)

In [ ]:
response.citations